In [1]:
import numpy as np
import cv2
import os
from ultralytics import YOLO

In [2]:
"""Extract annotations from annotations_file_path and outputs it in out dir in yolo format"""

import os.path
from xml.dom import minidom

out_dir = "./out"
annotations_file_path = "./annotations.xml"

# Constants
label_dict = {"cross":0,"circle":1,"triangle":2,"square":3}
bbox_type = tuple[int, int, int, int]

def main():
    # create out dir
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    file = minidom.parse(annotations_file_path)
    images = file.getElementsByTagName("image")

    for image in images:
        width = int(image.getAttribute("width"))
        height = int(image.getAttribute("height"))
        name = os.path.splitext(image.getAttribute("name"))[0]  # first ele is file name
        bboxes = image.getElementsByTagName("box")

        with open(os.path.join(out_dir, name + ".txt"), "w") as label_file:
            # iterate over pairs of bbox and keypoints for every sperm
            for bbox in bboxes:
                class_index = label_dict[bbox.getAttribute("label")]

                xtl, ytl, xbr, ybr = prepare_bbox(bbox)

                w = xbr - xtl
                h = ybr - ytl
                x_cen_norm = (xtl + (w / 2)) / width
                y_cen_norm = (ytl + (h / 2)) / height
                w_norm = w / width
                h_norm = h / height

                dataset_label = f"{class_index} {x_cen_norm} {y_cen_norm} {w_norm} {h_norm} "  # last space in str is necessary

                dataset_label = dataset_label.rstrip() + "\n"
                label_file.write(dataset_label)

    print("Finished writing all label files.")


def prepare_bbox(bbox: minidom.Element) -> bbox_type:
    xtl = int(float(bbox.getAttribute("xtl")))
    ytl = int(float(bbox.getAttribute("ytl")))
    xbr = int(float(bbox.getAttribute("xbr")))
    ybr = int(float(bbox.getAttribute("ybr")))
    return xtl, ytl, xbr, ybr

In [3]:
# main()

In [4]:
images_dir = "data/images/train/"

In [5]:
model = YOLO("runs/detect/train3/weights/last.pt")
shape_score_dict = {"circle":20,"square":15,"triangle":10,"cross":5}


In [6]:
def calculate_score(results):
    score = 0
    shapes_conversion_dict = results.names
    for obj in map(int,results.cpu().boxes.cls.int()):
        obj = int(obj)
        shape = shapes_conversion_dict[obj]
        score += shape_score_dict[shape]
    return score


In [7]:
from statistics import mode

In [8]:
cap = cv2.VideoCapture("rtsp://192.168.1.102:8554/unicast")
scores = []
while True:
    ret, frame = cap.read()
    results = model(frame,conf=0.5,show=False)
    scores.append(calculate_score(results[0]))
    if len(scores) == 50:
        scores.pop(0)
    score = np.median(scores)
    img = results[0].plot(labels=False)
    cv2.putText(img, f"Score: {score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('det',img)
    if cv2.waitKey(1) == ord('d'):
        break
cv2.destroyAllWindows()
cap.release()   

[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a start chunk; dropping frame.
[rtsp @ 0x55b961dfb080] Received packet without a st

: 

In [20]:
def black_detection(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    lower_black = np.array([0, 0, 0], dtype=np.uint8)
    upper_black = np.array([255, 200, 255], dtype=np.uint8)
    black_mask = cv2.inRange(hsv, lower_black, upper_black)
    # black_detected = cv2.bitwise_and(img, img, mask=black_mask)
    return black_mask

In [21]:
for img_name in os.listdir("shapes_pics"):
    img_path = os.path.join("shapes_pics",img_name)
    img = cv2.imread(img_path)
    # crop left half of image
    img = img[65:-60,int(img.shape[1]/2)+150:]
    orig = np.array(img)
    cv2.imwrite(os.path.join("out",img_name),img)
    # img = black_detection(img)
#     cv2.imshow('orig',cv2.resize(orig,None,fx=0.5,fy=0.5))
#     # cv2.imshow('img',cv2.resize(img,None,fx=0.5,fy=0.5))
#     if cv2.waitKey(0)&0xff == ord('d'):
#         break
# cv2.destroyAllWindows()

In [22]:
# capture atef's video stream
cap = cv2.VideoCapture("rtsp://:8080/unicast")
while True:
    ret, frame = cap.read()
    frame = black_detection(frame)
    cv2.imshow('det',frame)
    if cv2.waitKey(1) == ord('d'):
        break
cv2.destroyAllWindows()
cap.release()   

[tcp @ 0x55bfeb822b00] Connection to tcp://10.10.221.34:8080 failed: No route to host
[ERROR:0@1004.016] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): https://10.10.221.34:8080/unicast in function 'icvExtractPattern'




error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
